# Transform

Testing ground for service call data transformation

In [1]:
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 10:10:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pq_path = Path("../tests/resources/SR2020.parquet")
df = spark.read.parquet(str(pq_path))
df.printSchema()

root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)



In [ ]:
# stop current SparkContext before creating a new one
spark.stop()

Read from cloud storage using `gcs-connector-hadoop3` jar

Before starting a new context, I had to restart the kernel for spark to recognize the gcs-connector jar

In [31]:
!mkdir ../data/lib
!gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar  \
    ../data/lib/gcs-connector-hadoop3-latest.jar  
# !curl -O https://github.com/GoogleCloudDataproc/hadoop-connectors/releases/download/v2.2.11/gcs-connector-hadoop3-2.2.11-shaded.jar
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [2]:
conf = (
    SparkConf()
    .setMaster("local[*]")
    .setAppName("test_cloud")
    .set("spark.jars", "../data/lib/gcs-connector-hadoop3-latest.jar")
)
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set(
    "fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
)
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()

23/03/21 13:41:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
DATA_LAKE = os.getenv("DATA_LAKE")
gcs_path = f"gs://{DATA_LAKE}/raw/pq/sr2023.parquet"
print(f"gcs_path: {gcs_path}")
df_gcs = spark.read.parquet(gcs_path)
df_gcs.printSchema()

gcs_path: gs://service-calls-data-lake/raw/pq/sr2023.parquet


root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)



## Transformations

How should we model our service call data?

Most important fields:

- Service Request Type
- First 3 Chars of Postal Code
- ward_name
- datetime

Ideas:

- most/least common types per month?
- most/least common types by ward/FSA?
- highest frequency ward/FSA by service types, i.e. which ward has the most/least amount of noise complaints?
- combine with population per FSA and find requests per capita?
- distributions of the all/top 80% of request types
    - filter by ward/FSA/season?

Division and section could be queried against to show which municipal entity receive the most requests, although that would highly correlate with the type of requests

In [5]:
import folium

In [6]:
# for testing
gcs_path = f"gs://{DATA_LAKE}/raw/pq/SR2021.parquet"
df = spark.read.parquet(gcs_path)
print(f"row count: {df.count()}")

row count: 323880


In [16]:
try:
    df.corr("Service Request Type", "Division")
except Exception as e:
    print(e)

requirement failed: Currently correlation calculation for columns with dataType string not supported.


In [25]:
from pyspark.sql.functions import col

# request type count by ward
df_ward_type = (
    df.filter(df.ward_name.isNotNull())
    .select("creation_datetime", "ward_name", "Service Request Type")
    .groupBy(["ward_name", "Service Request Type"])
    .count()
)
# most common request type's count by ward
df_ward_max = (
    df_ward_type.alias("df_ward_max")
    .groupBy("ward_name")
    .max("count")
    .withColumnRenamed("max(count)", "max_count")
)

# trying to join them get get most common type by ward
df_ward = (
    df_ward_max.join(df_ward_type, col("max_count") == col("count"), "inner")
    .select("df_ward_max.ward_name", "Service Request Type", "count")
    .orderBy("ward_name")
)
df_ward.show(n=30, truncate=40)

+------------------------+---------------------------------------+-----+
|               ward_name|                   Service Request Type|count|
+------------------------+---------------------------------------+-----+
|       Beaches-East York|                       CADAVER WILDLIFE|  788|
|               Davenport|Residential: Bin: Repair or Replace Lid| 1141|
|         Don Valley East|                     Property Standards|  349|
|        Don Valley North|                                 Zoning|  413|
|        Don Valley North|Residential: Bin: Repair or Replace Lid|  413|
|         Don Valley West|                        General Pruning|  575|
|       Eglinton-Lawrence|Residential: Bin: Repair or Replace Lid|  984|
|        Etobicoke Centre|                        General Pruning|  845|
|         Etobicoke North|                     Property Standards|  517|
|     Etobicoke-Lakeshore|                        General Pruning|  913|
|Humber River-Black Creek|                     Prop

Seeing some repeats of ward name due to counts of different request types being equal; is that a coincidence or is that really true?